In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import random
import statsmodels.formula.api as sm
import matplotlib.pyplot as plt

In [2]:
#columns_to_keep = ['hfoodsp','hfdval','h_numper','h_seq','a_age','marsupwt','moop','hhi_yn','chsp_val',
                  # 'gestfips','pedisout','pedisdrs','pedisear', 'pedisrem','pediseye', 'pedisphy', 
                  # 'vet_typ1','vet_yn','pemlr','filestat','wsal_val','semp_val','frse_val',
                  # 'ss_val','rtm_val','oi_val','oi_off','int_yn','uc_yn', 'uc_val','int_val','hfoodmo']

In [3]:
CPS_dataset = pd.read_csv('C:\Users\wangy\OneDrive\Documents\BasicIncomeProject\SNAP.csv')

In [4]:
#recipient or not
CPS_dataset.hfoodsp = np.where(CPS_dataset.hfoodsp == "Yes",1,0)

In [5]:
temp = CPS_dataset.groupby('h_seq')['hfoodsp']

In [6]:
household_SNAP = CPS_dataset.groupby('h_seq')['hfoodsp'].mean()
household_size = CPS_dataset.groupby('h_seq')['h_numper'].mean()

In [7]:
#earned income
wage = pd.to_numeric(np.where(CPS_dataset.wsal_val!= 'None or not in universe', CPS_dataset.wsal_val, 0))
self_employed1 = pd.to_numeric(np.where(CPS_dataset.semp_val!= 'None or not in universe', CPS_dataset.semp_val, 0))
self_employed2 = pd.to_numeric(np.where(CPS_dataset.frse_val!= 'None or not in universe', CPS_dataset.frse_val, 0))
p_earned = wage + self_employed1 + self_employed2 #individual earned income
p_earned = 0.8 * p_earned #for net income calculation, there is a 20% deduction in earned income
CPS_dataset['p_earned'] = p_earned

In [8]:
#unearned income
ss = pd.to_numeric(np.where(CPS_dataset.ss_val!='None or not in universe', CPS_dataset.ss_val, 0))
pension = pd.to_numeric(np.where(CPS_dataset.rtm_val!='None or not in universe', CPS_dataset.rtm_val, 0))
disability = pd.to_numeric(np.where(CPS_dataset.oi_off=='State disability payments', CPS_dataset.oi_val, 0))
unemploy = pd.to_numeric(np.where(CPS_dataset.uc_yn=='Yes', CPS_dataset.uc_val, 0))
interest = pd.to_numeric(np.where(CPS_dataset.int_yn=='Yes', CPS_dataset.int_val, 0))
p_unearned = ss + pension + disability + unemploy + interest #individual unearned income
CPS_dataset['p_unearned'] = p_unearned

In [9]:
CPS_dataset['hh_net_income'] = p_earned + p_unearned
hh_net_income = CPS_dataset.groupby('h_seq')['hh_net_income'].sum()

In [10]:
hh_SNAP = DataFrame(household_SNAP.transpose())

In [11]:
hh_SNAP['hh_net'] = hh_net_income
hh_SNAP['hh_size'] = household_size

In [12]:
hh_SNAP.columns = ['indicator', 'hh_net','hh_size'] #indicator is hfoodsp
# print hh_SNAP

In [13]:
#net income deduction
hh_SNAP.hh_net = np.where(hh_SNAP.hh_size <=3, hh_SNAP.hh_net-155*12, hh_SNAP.hh_net)
hh_SNAP.hh_net = np.where(hh_SNAP.hh_size ==4, hh_SNAP.hh_net-168*12, hh_SNAP.hh_net)
hh_SNAP.hh_net = np.where(hh_SNAP.hh_size ==5, hh_SNAP.hh_net-197*12, hh_SNAP.hh_net)
hh_SNAP.hh_net = np.where(hh_SNAP.hh_size >=6, hh_SNAP.hh_net-226*12, hh_SNAP.hh_net)

In [14]:
#medical deduction
#age over 60
CPS_dataset.a_age = np.where(CPS_dataset.a_age == "80-84 years of age",
                             random.randrange(80, 84),
                             CPS_dataset.a_age)
CPS_dataset.a_age = np.where(CPS_dataset.a_age == "85+ years of age",
                             random.randrange(85, 95),
                             CPS_dataset.a_age)
CPS_dataset.a_age = pd.to_numeric(CPS_dataset.a_age)
#disabled
CPS_dataset['disability'] = np.zeros(len(CPS_dataset))
CPS_dataset.disability = np.where(CPS_dataset.pedisdrs == 'Yes', 1, CPS_dataset.disability)
CPS_dataset.disability = np.where(CPS_dataset.pedisear == 'Yes', 1, CPS_dataset.disability)
CPS_dataset.disability = np.where(CPS_dataset.pediseye == 'Yes', 1, CPS_dataset.disability)
CPS_dataset.disability = np.where(CPS_dataset.pedisout == 'Yes', 1, CPS_dataset.disability)
CPS_dataset.disability = np.where(CPS_dataset.pedisphy == 'Yes', 1, CPS_dataset.disability)
CPS_dataset.disability = np.where(CPS_dataset.pedisrem == 'Yes', 1, CPS_dataset.disability)
#deduction of more than $35 for a month
CPS_dataset.hhi_yn = np.where(CPS_dataset.hhi_yn == 'Yes', 1, 0)
CPS_dataset.moop = np.where(CPS_dataset.moop != 'NIU', CPS_dataset.moop, 0)
CPS_dataset['medical'] = np.zeros(len(CPS_dataset))
CPS_dataset.medical = np.where((CPS_dataset.hhi_yn == 0), CPS_dataset.moop, 0)
CPS_dataset.medical = np.where((CPS_dataset.disability == 1.0)|(CPS_dataset.a_age >= 60), CPS_dataset.moop, 0)
CPS_dataset.medical = pd.to_numeric(CPS_dataset.medical)
hh_medical = CPS_dataset.groupby('h_seq')['medical'].sum()
hh_SNAP['hh_medical'] = hh_medical
hh_SNAP.hh_net = np.where(hh_medical > 35*12, hh_SNAP.hh_net-(hh_medical-35*12), hh_SNAP.hh_net)

In [15]:
#child support
CPS_dataset.chsp_val = np.where(CPS_dataset.chsp_val == "NIU", 0, CPS_dataset.chsp_val)
CPS_dataset.chsp_val = pd.to_numeric(CPS_dataset.chsp_val)
hh_child = CPS_dataset.groupby('h_seq')['chsp_val'].sum()
hh_SNAP['hh_child'] = hh_child
hh_SNAP.hh_net = hh_SNAP.hh_net - hh_SNAP.hh_child

In [16]:
hh_SNAP.hh_net = hh_SNAP.hh_net - (10+290)*12 #dependent care; excess shelter. All based on SNAP official average
hh_SNAP.hh_net = np.where(hh_SNAP.hh_net <0, 0, hh_SNAP.hh_net)

In [17]:
hh_SNAP.hh_net = np.where(hh_SNAP.hh_net > 5490 * 12, -100, hh_SNAP.hh_net)
hh_SNAP = hh_SNAP[hh_SNAP.hh_net>=0]

In [18]:
model = sm.ols(formula='indicator ~ hh_net', data=hh_SNAP).fit()
print model.summary()

                            OLS Regression Results                            
Dep. Variable:              indicator   R-squared:                       0.121
Model:                            OLS   Adj. R-squared:                  0.121
Method:                 Least Squares   F-statistic:                     5187.
Date:                Sat, 12 Nov 2016   Prob (F-statistic):               0.00
Time:                        21:47:46   Log-Likelihood:                -13579.
No. Observations:               37780   AIC:                         2.716e+04
Df Residuals:                   37778   BIC:                         2.718e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      0.3364      0.003    112.407      0.0

In [19]:
probs = model.fittedvalues
probs

h_seq
5        0.227720
8        0.195499
19      -0.084678
27       0.083350
28       0.284077
29       0.065933
31       0.336381
34       0.137647
37       0.200279
39       0.336381
40       0.161096
52       0.259342
53       0.238492
58       0.166307
63       0.184632
69       0.018640
71       0.293570
72       0.073826
74       0.239542
75       0.174803
76       0.098457
77       0.206311
84       0.050899
85       0.149428
88       0.266256
91       0.289120
92       0.140723
93       0.336381
94       0.336381
95       0.225310
           ...   
97847   -0.092071
97848    0.080657
97850    0.300033
97851    0.077028
97852    0.012164
97855    0.308509
97856    0.005772
97875    0.212612
97880   -0.008521
97882   -0.011409
97883    0.336381
97884   -0.062375
97892    0.093711
97895    0.066148
97896    0.224603
97899    0.061133
97902    0.312151
97903    0.039562
97904    0.185151
97908   -0.064777
97909   -0.078518
97910    0.118008
97913    0.276183
97914    0.176863
9791

In [20]:
#weights; states; SNAP value
household_marsupwt = CPS_dataset.groupby('h_seq')['marsupwt'].mean()
hh_SNAP['hh_marsupwt'] = household_marsupwt

household_gestfips = CPS_dataset.groupby('h_seq')['gestfips'].mean()
hh_SNAP['hh_gestfips'] = household_gestfips

CPS_dataset.hfdval = np.where(CPS_dataset.hfdval!= 'Not in universe', CPS_dataset.hfdval, 0)
CPS_dataset.hfdval = pd.to_numeric(CPS_dataset.hfdval)
household_hfdval = CPS_dataset.groupby('h_seq')['hfdval'].mean()
hh_SNAP['hh_hfdval'] = household_hfdval


In [21]:
#Monthly weight of SNAP preparation
CPS_dataset.hfoodmo = np.where(CPS_dataset.hfoodmo == 'Not in universe', 0, CPS_dataset.hfoodmo)
CPS_dataset.hfoodmo = np.where(CPS_dataset.hfoodmo == '12 Months', 12, CPS_dataset.hfoodmo)
CPS_dataset.hfoodmo = np.where(CPS_dataset.hfoodmo == '1 month', 1, CPS_dataset.hfoodmo)
CPS_dataset.hfoodmo = pd.to_numeric(CPS_dataset.hfoodmo)
household_hfoodmo = CPS_dataset.groupby('h_seq')['hfoodmo'].mean()
hh_SNAP['hh_hfoodmo'] = household_hfoodmo

In [22]:
hh_SNAP

,indicator,hh_net,hh_size,hh_medical,hh_child,hh_marsupwt,hh_gestfips,hh_hfdval,hh_hfoodmo
h_seq,,,,,,,,,
5,0,16140.0,1,200,0,698.490000,23,0,0
8,0,20926.0,2,5100,0,982.400000,23,0,0
19,0,62542.0,2,0,0,943.990000,23,0,0
27,0,37584.0,4,0,0,475.190000,23,0,0
28,1,7769.0,1,450,0,1085.380000,23,1344,12
29,0,40171.0,2,2320,0,937.235000,23,0,0
31,1,0.0,1,360,0,616.720000,23,1812,12
34,0,29519.0,2,900,0,982.620000,23,0,0
37,0,20216.0,1,0,4970,1181.960000,23,0,0


In [23]:
admin = pd.read_csv('C:\Users\wangy\OneDrive\Documents\BasicIncomeProject\Administrative.csv',
                    dtype={ 'Household number': np.float,'Average household benefit': np.float, 'Total': np.float, 'Individual number': np.float})
#dtype={'Household number': np.float,'Average household benefit': np.float, 'Total': np.float}
admin.index = admin.Fips

In [24]:
# CPS total benefits and Administrative total benefits
state_benefit = {}
state_recipients = {}
for state in admin.Fips:
    this_state = (hh_SNAP.hh_gestfips==state)
    CPS_totalb = (hh_SNAP.hh_hfdval * hh_SNAP.hh_marsupwt / hh_SNAP.hh_hfoodmo)[this_state].sum()/1000000 #in million, per month
    admin_totalb =  admin.Total[state] /12 /1000000 # in million, per month
    CPS_totaln = (hh_SNAP.hh_marsupwt[this_state&hh_SNAP.indicator==1]*hh_SNAP.hh_hfoodmo/12).sum()/1000 # in thousand, per month
    admin_totaln =  admin["Household number"][state] /1000 #in thousand, per month
    
    temp = [admin.State[state], CPS_totalb, admin_totalb, CPS_totaln, admin_totaln]
    state_benefit[state] = temp
    
pre_augment_benefit = DataFrame(state_benefit).transpose()
pre_augment_benefit.columns = ['State', 'CPS total benefits(monthly)','Admin total benefits(monthly)',
                               'CPS total household recipient(monthly)','Admin total household recipient(monthly)']

In [25]:
pre_augment_benefit

,State,CPS total benefits(monthly),Admin total benefits(monthly),CPS total household recipient(monthly),Admin total household recipient(monthly)
1,Alabama,72.3154,109.844,225.58,419.552
2,Alaska,7.90355,14.5202,20.0439,36.9972
4,Arizona,119.96,123.063,327.805,449.64
5,Arkansas,39.731,55.3099,151.897,221.446
6,California,322.966,617.624,964.365,2019.27
8,Colorado,46.3731,63.8114,142.968,234.098
9,Connecticut,41.3462,58.1196,131.297,242.982
10,Delaware,13.5626,18.3627,39.8634,71.408
11,District of Columbia,10.0727,18.5503,38.2384,80.3602
12,Florida,238.699,456.07,814.854,1920.7


In [26]:
pre_augment_benefit.to_csv('C:\Users\wangy\OneDrive\Documents\BasicIncomeProject\pre-blow-up.csv')

In [27]:
# caculate difference of SNAP stats and CPS aggregates on recipients number
# by state
diff = {'Fips':[],'Difference in Population':[],'Mean Benefit':[],'CPS Population':[],'SNAP Population':[]}
diff['Fips'] = admin.Fips
current = (hh_SNAP.indicator==1)
for FIPS in admin.Fips:
        this_state = (hh_SNAP.hh_gestfips==FIPS)
        current_tots = (hh_SNAP.hh_marsupwt[current&this_state]*hh_SNAP.hh_hfoodmo/12).sum()
        valid_num = (hh_SNAP.hh_marsupwt[current&this_state]*hh_SNAP.hh_hfoodmo/12).sum() + 0.0000001
        current_mean = ((hh_SNAP.hh_hfdval * hh_SNAP.hh_marsupwt / hh_SNAP.hh_hfoodmo)[current&this_state].sum())/valid_num
        diff['CPS Population'].append(current_tots)
        diff['SNAP Population'].append(float(admin["Household number"][admin.Fips == FIPS]))
        diff['Difference in Population'].append(float(admin["Household number"][admin.Fips == FIPS])- current_tots)
        diff['Mean Benefit'].append(current_mean)

In [28]:
d = DataFrame(diff)
d = d[['Fips', 'Mean Benefit', 'Difference in Population', 'CPS Population', 'SNAP Population']]
d.to_csv('recipients.csv', index=False)

In [29]:
hh_SNAP['impute'] = np.zeros(len(hh_SNAP))
hh_SNAP['snap_impute'] = np.zeros(len(hh_SNAP))

non_current = (hh_SNAP.indicator==0)
current = (hh_SNAP.indicator==1)
random.seed()

for FIPS in admin.Fips:
    
        print ('we need to impute', d['Difference in Population'][FIPS], 'for state', FIPS)
        
        if d['Difference in Population'][FIPS] < 0:
            continue
        else:
            this_state = (hh_SNAP.hh_gestfips==FIPS)
           
            not_imputed = (hh_SNAP.impute==0)
            
            pool_index = hh_SNAP[this_state&not_imputed&non_current].index
            pool = DataFrame({'weight': hh_SNAP.hh_marsupwt[pool_index], 'prob': probs[pool_index]},
                            index=pool_index)
            
            pool = pool.sort(columns='prob', ascending=False)
            pool['cumsum_weight'] = pool['weight'].cumsum()
            pool['distance'] = abs(pool.cumsum_weight-d['Difference in Population'][FIPS])
            min_index = pool.sort(columns='distance')[:1].index
            min_weight = int(pool.loc[min_index].cumsum_weight)
            pool['impute'] = np.where(pool.cumsum_weight<=min_weight+10 , 1, 0)
            hh_SNAP.impute[pool.index[pool['impute']==1]] = 1
            hh_SNAP.snap_impute[pool.index[pool['impute']==1]] = admin['Average household benefit'][FIPS]*12

        print ('Method1: regression gives', 
                hh_SNAP.hh_marsupwt[(hh_SNAP.impute==1)&this_state].sum()) 

('we need to impute', 193971.92316468252, 'for state', 1)
('Method1: regression gives', 193027.59261904753)
('we need to impute', 16953.308549603178, 'for state', 2)
('Method1: regression gives', 16797.314166666667)
('we need to impute', 121834.77696626983, 'for state', 4)


C:\Users\wangy\Anaconda2\lib\site-packages\ipykernel\__main__.py:23: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
C:\Users\wangy\Anaconda2\lib\site-packages\ipykernel\__main__.py:26: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
C:\Users\wangy\Anaconda2\lib\site-packages\ipykernel\__main__.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\wangy\Anaconda2\lib\site-packages\ipykernel\__main__.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


('Method1: regression gives', 119766.6946666667)
('we need to impute', 69548.836071428552, 'for state', 5)
('Method1: regression gives', 69475.75000000001)
('we need to impute', 1054907.1063809535, 'for state', 6)
('Method1: regression gives', 1057804.2596245422)
('we need to impute', 91129.277058531821, 'for state', 8)
('Method1: regression gives', 91897.0976190476)
('we need to impute', 111685.27889285717, 'for state', 9)
('Method1: regression gives', 110547.62683333334)
('we need to impute', 31544.582886904755, 'for state', 10)
('Method1: regression gives', 31343.850833333338)
('we need to impute', 42121.886740079382, 'for state', 11)
('Method1: regression gives', 42309.51083333331)
('we need to impute', 1105843.3499348853, 'for state', 12)
('Method1: regression gives', 1106487.1476666662)
('we need to impute', 478369.01947222208, 'for state', 13)
('Method1: regression gives', 477570.1771190478)
('we need to impute', 52185.792585317453, 'for state', 15)
('Method1: regression gives',

In [30]:
results = {}

imputed = (hh_SNAP.impute == 1)
has_val = (hh_SNAP.hh_hfdval != 0)
no_val = (hh_SNAP.hh_hfdval == 0)

for FIPS in admin.Fips:
    this_state = (hh_SNAP.hh_gestfips==FIPS)
    
    current_total = (hh_SNAP.hh_hfdval * hh_SNAP.hh_marsupwt)[this_state].sum() #yearly benefit
    imputed_total = (hh_SNAP.snap_impute * hh_SNAP.hh_marsupwt)[this_state&imputed].sum()
    on_file = current_total + imputed_total

    admin_total = admin.Total[FIPS]
    
    adjust_ratio = admin_total / on_file
    this_state_num = [admin['State'][FIPS], on_file, admin_total, adjust_ratio]
    results[FIPS] = this_state_num
    

    hh_SNAP.snap_impute = np.where(has_val&this_state, hh_SNAP.hh_hfdval * adjust_ratio, hh_SNAP.snap_impute)
    hh_SNAP.snap_impute = np.where(no_val&this_state, hh_SNAP.snap_impute * adjust_ratio, hh_SNAP.snap_impute)

hh_SNAP["snap_participation"] = np.zeros(len(hh_SNAP))
hh_SNAP["snap_participation"] = np.where(hh_SNAP.impute==1, 2, 0)
hh_SNAP["snap_participation"] = np.where(has_val, 1, hh_SNAP.snap_participation)


r = DataFrame(results).transpose()
r.columns=['State', 'Imputed', 'Admin', 'adjust ratio']
r.to_csv('amount.csv', index=False)

In [31]:
hh_SNAP.to_csv('SNAP_Imputation.csv', 
                   columns=['snap_participation', 'snap_impute'])

In [32]:
#Individual check
imputed = (hh_SNAP.snap_participation != 0)
result = {}

for FIPS in admin.Fips:
    this_state = (hh_SNAP.hh_gestfips==FIPS)
    admin_totalindividual =  admin["Individual number"][FIPS]
    imputed_totalindividual = ( hh_SNAP.hh_marsupwt* hh_SNAP.hh_size)[this_state&imputed].sum()
    this_state_numindividual = [admin['State'][FIPS], admin_totalindividual, imputed_totalindividual]
    results[FIPS] = this_state_numindividual
    
r = DataFrame(results).transpose()
r.columns=['State', 'admin_totalindividual', 'imputed_totalindividual']
r.to_csv('check_individual.csv', index=False)